# 첨부 가져오기

In [0]:
import pandas as pd
pdf_file = pd.read_csv('./※ 인도 시장_75QNED8_영상깜빡임_250708.csv')[['Set', 'MAC_ADDRESS']]
pdf_file['mac_addr_2'] = pdf_file['MAC_ADDRESS'].str[:2]+':'+\
                         pdf_file['MAC_ADDRESS'].str[2:4]+':'+\
                         pdf_file['MAC_ADDRESS'].str[4:6]+':'+\
                         pdf_file['MAC_ADDRESS'].str[6:8]+':'+\
                         pdf_file['MAC_ADDRESS'].str[8:10]+':'+\
                         pdf_file['MAC_ADDRESS'].str[10:12]
pdf_file = pdf_file.reset_index(drop=False)
pdf_file

In [0]:
# spark dataframe 변환
df_file= spark.createDataFrame(pdf_file)
df_file.display()

In [0]:
# 컬럼 추가
from pyspark.sql.functions import sha2, col, when, lit, concat

tv_salt = dbutils.secrets.get("admin", "salt")

df_file = df_file.withColumn("Set_hashed", 
                               when(col("Set").isNull() | (col("Set") == ''), 
                                    col("Set"))\
                                .otherwise(sha2(concat(col("Set"), lit(tv_salt)), 256)))
df_file = df_file.withColumn("MAC_ADDRESS_hashed", 
                               when(col("mac_addr_2").isNull() | (col("MAC_ADDRESS") == ''), 
                                    col("mac_addr_2"))\
                                .otherwise(sha2(concat(col("mac_addr_2"), lit(tv_salt)), 256)))
df_file.display() 

# 데이터 가져오기

In [0]:
%python 
df_min = spark.sql(''' 
    select mac_addr, min(crt_date) as min_crt_date
    from   kic_data_ods.tlamp.activation_date
    group by mac_addr
''')
df_min.display()

# 결합

In [0]:
df_join = df_file.join(df_min, 
                       on=[df_min['mac_addr'] == df_file['MAC_ADDRESS_hashed']],
                       how='left')

In [0]:
df_join.columns

In [0]:
df_join.select("index", "Set_hashed", "MAC_ADDRESS_hashed", 'min_crt_date').display()